## PMFs

In [2]:
import thinkstats2
pmf = thinkstats2.Pmf([1,2,2,3,5])
pmf

Pmf({1: 0.2, 2: 0.4, 3: 0.2, 5: 0.2})

`Hist`: maps from values to integer counters

`pmf`: maps from values to floating point probabilities

## Plotting PMFs